In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from dnn_training.dataset import *
import numpy as np
from pathlib import Path

In [3]:
device = "cpu"

In [4]:
inputFolder = "/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v15-sampleDump/superclsDumper_201.root"
dataset_ak = makeTargetBinary(selectSeedOnly(zipDataset(loadDataset_ak(inputFolder))))
dataset_ak

<Array [[{run_: 1, ...}, ..., {...}], ...] type='100 * var * inferencePair[...'>

In [18]:
ak.broadcast_arrays(ak.local_index(dataset_ak.seedTracksterIdx, axis=0), dataset_ak.seedTracksterIdx)[0]

<Array [[0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0], ...] type='100 * var * int64'>

In [30]:
len(ak.flatten(dataset_ak.seedTracksterIdx))

11144

In [31]:
np.arange(len(ak.flatten(dataset_ak.seedTracksterIdx)))

array([    0,     1,     2, ..., 11141, 11142, 11143])

In [33]:
ak.num(dataset_ak.seedTracksterIdx, axis=1)

<Array [41, 228, 127, 63, 39, 63, ..., 188, 12, 95, 70, 96] type='100 * int64'>

In [ ]:
ak.unflatten(np.arange(len(ak.flatten(dataset_ak.seedTracksterIdx))), )

In [38]:
dataset_ak.seedTracksterBestAssociation_caloParticleEnergy[:, 0]

<Array [290, 169, 419, 74.4, ..., 140, 313, 89.9, 441] type='100 * float32'>

In [36]:
dataset_ak.seedTracksterBestAssociation_caloParticleEnergy[ak.unflatten(ak.local_index(ak.flatten(dataset_ak.seedTracksterIdx), axis=0), ak.num(dataset_ak.seedTracksterIdx, axis=1))[:, 0]]

IndexError: cannot slice ListOffsetArray (of length 100) with array([    0,    41,   269,   396,   459,   498,   561,   629,   659,
         708,   758,   814,   879,   951,  1052,  1069,  1200,  1260,
        1282,  1431,  1491,  1718,  1916,  2115,  2147,  2170,  2245,
        2393,  2429,  2526,  2602,  2644,  2718,  2745,  2959,  3114,
        3156,  3229,  3418,  3666,  3929,  3996,  4062,  4091,  4147,
        4314,  4362,  4399,  4500,  4706,  4975,  5192,  5378,  5632,
        5671,  5919,  5963,  6106,  6119,  6200,  6406,  6580,  6616,
        6659,  6889,  6922,  7204,  7298,  7326,  7381,  7469,  7502,
        7560,  7740,  7800,  7858,  8090,  8312,  8523,  8560,  8630,
        8657,  8754,  8822,  8842,  8984,  9209,  9256,  9452,  9539,
        9762, 10017, 10101, 10282, 10527, 10683, 10871, 10883, 10978,
       11048]): index 269 is out of bounds for axis 0 with size 100

In [14]:
datasets = makeDatasetsTrainVal(dataset_ak) #makeDatasetsTrainVal_fromCache("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v13/superclsDumper_*.root") # makeDatasetsTrainVal(dataset_ak)
val_data = datasets["valDataset"]
val_data

{'features': tensor([[ 1.3186e-01,  6.3233e-01,  1.7943e+01,  ...,  3.6786e-01,
           3.7236e+01,  9.7225e-01],
         [ 1.4764e-01, -4.6271e-01,  1.5315e+01,  ...,  7.3828e-01,
           2.1150e+01,  9.7144e-01],
         [-3.5058e-02,  2.9497e+00,  1.2208e+01,  ...,  4.9755e-01,
           1.7601e+01,  9.9592e-01],
         ...,
         [ 8.3408e-02,  6.7571e-01,  3.5812e+00,  ...,  1.6211e-01,
           2.9612e+01,  9.6364e-01],
         [ 9.8238e-02,  1.8090e+00,  2.0153e+00,  ...,  9.9885e-02,
           1.8380e+01,  9.2932e-01],
         [ 1.4009e-02,  1.7524e+00,  2.6815e+00,  ...,  1.5979e-01,
           2.1179e+01,  9.7526e-01]]),
 'genmatching': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'seedTracksterBestAssociationScore': tensor([0.0005, 0.0005, 0.0005,  ..., 0.0031, 0.0031, 0.0031]),
 'candidateTracksterAssociationWithSeed_score': tensor([1., 1., 1.,  ..., 1., 1., 1.]),
 'caloParticleEnergy': tensor([210.7941, 210.7941, 210.7941,  ..., 339.8661, 339.8661, 339.8661]

In [7]:
val_data["features"][:, 2].shape

torch.Size([2054])

In [11]:
pred = torch.rand(val_data["features"][:, 2].shape[0])
pred

tensor([0.8551, 0.3361, 0.2086,  ..., 0.8639, 0.2475, 0.3683])

In [12]:
pred < 0.5

tensor([False,  True,  True,  ..., False,  True,  True])

In [13]:
torch.scatter_add(torch.zeros(torch.max(val_data["eventIndex"])+1), 0, val_data["eventIndex"], val_data["features"][:, 2]*(pred < 0.5))

tensor([105.5135,  75.3709, 370.6978, 730.5784, 114.5897, 135.4581,  41.3484,
         45.3838, 171.1164, 551.3284, 600.3002, 157.6989, 116.6326,  22.6670,
        221.9060, 139.1373,  50.8158, 236.6286, 502.5782, 409.4410])

{'trainDataset': <torch.utils.data.dataset.StackDataset at 0x7f2b07bbaa10>,
 'valDataset': {'features': tensor([[ 1.1266e-01, -4.4898e+00,  1.6038e+01,  ...,  2.4633e-02,
            2.0613e+01,  9.6328e-01],
          [ 6.2158e-03,  4.7919e-02,  1.5324e+01,  ...,  9.7106e-03,
            2.6007e+01,  9.8964e-01],
          [ 1.0266e-01, -4.3172e+00,  1.2841e+01,  ...,  3.6511e-02,
            4.2363e+01,  9.8959e-01],
          ...,
          [ 8.1457e-02,  1.7278e+00,  1.6705e+00,  ...,  1.4557e-01,
            9.4672e+00,  9.5305e-01],
          [ 8.0575e-02,  2.3707e+00,  2.5536e+00,  ...,  1.7865e-01,
            1.8592e+01,  9.7720e-01],
          [ 5.8992e-02, -2.8771e-01,  3.3665e+00,  ...,  1.7304e-01,
            3.0193e+01,  9.8598e-01]]),
  'genmatching': tensor([0., 1., 0.,  ..., 0., 0., 0.]),
  'seedTracksterBestAssociationScore': tensor([0.0016, 0.0016, 0.0016,  ..., 0.0005, 0.0005, 0.0005]),
  'candidateTracksterAssociationWithSeed_score': tensor([1.0000, 0.0032, 1.0000

In [24]:
ak.to_numpy(ak.flatten(ak.broadcast_arrays(ak.local_index(dataset_ak.seedTracksterIdx, axis=0), dataset_ak.seedTracksterIdx)[0]))

array([ 0,  0,  0, ..., 99, 99, 99])

In [7]:
dataset_torch[0:5]

{'features': (tensor([[ 1.4991e-01,  1.6956e+00,  2.3841e+01, -2.8661e+00,  2.7054e+00,
           -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.1916e+01,  2.8193e-01,
            1.3713e-01,  1.3713e-01,  1.8231e+00,  5.3159e-02,  2.7881e-01,
            4.4138e+01,  9.7236e-01],
          [ 5.2284e-02,  1.1760e+00,  2.6361e+01, -2.7684e+00,  3.2958e+00,
           -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.8218e+01,  1.1699e-01,
            7.7512e-03,  7.7512e-03,  6.5022e-01,  1.0302e-01,  5.5533e-02,
            2.5784e+01,  9.3319e-01],
          [ 1.6820e-01,  5.6003e-01,  1.2826e+01, -2.8844e+00,  1.4293e+00,
           -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.0800e+01,  3.6202e-01,
            2.4707e-01,  2.4707e-01,  1.4768e+00,  2.1777e-01,  2.9864e-01,
            3.4246e+01,  9.2995e-01],
          [-8.6307e-02,  8.1876e-01,  1.6024e+01, -2.6298e+00,  2.2984e+00,
           -2.7162e+00,  6.6292e-01,  5.4562e+02,  7.8261e+01,  3.2822e-01,
            1.9353e-01,  1.9353e-01,  

In [22]:
val_data[:]["features"][0].nelement()

24164293

In [86]:
gen = train_data[:]["genmatching"][0]
sampler = WeightedRandomSampler(torch.where(gen==1, torch.sum(gen==0)/torch.sum(gen==1), 1.), gen.shape[0], replacement=True)

In [87]:
train_dataloader = DataLoader(train_data, batch_size=1024, shuffle=False, sampler=sampler)
train_iter = iter(train_dataloader)

In [88]:
len(gen)/1024

9.1015625

In [99]:
batch = next(train_iter)["genmatching"][0].numpy()
batch

array([0., 0., 1., ..., 0., 1., 0.], dtype=float32)

In [100]:
np.count_nonzero(batch), len(batch)

(498, 1024)

In [20]:
list(torch.where(gen==1, torch.sum(gen==0)/torch.sum(gen==1), 1.))

[tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),

In [3]:
torch.min(val_data[:]["features"][0][:, 2])

tensor(1.5004)

In [10]:
torch.sum(train_data[:]["genmatching"][0] == 0)

tensor(9174)

In [4]:
dataset_ak = loadDataset_ak("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v13/superclsDumper_1.root")

In [3]:
d_zipped = zipDataset(dataset_ak)

In [4]:
tgt = makeTargetBinary(selectSeedOnly(zipDataset(dataset_ak)))
tgt

<Array [[{...}, {...}, ..., {...}, {...}], ...] type='100 * var * inference...'>

In [7]:
features = ['DeltaEtaBaryc', 'DeltaPhiBaryc', 'multi_en', 'multi_eta', 'multi_pt', 'seedEta','seedPhi','seedEn', 'seedPt', 'theta', 'theta_xz_seedFrame', 'theta_yz_seedFrame', 'theta_xy_cmsFrame', 'theta_yz_cmsFrame', 'theta_xz_cmsFrame', 'explVar', 'explVarRatio']

In [8]:
feats_ak = [ak.to_numpy(ak.flatten(tgt["feature_" + featName])) for featName in features]

In [9]:
np.stack(feats_ak, axis=1).shape

(1881417, 17)

In [10]:
torch.stack([torch.tensor(ar) for ar in feats_ak])

tensor([[ 1.4991e-01,  5.2284e-02,  1.6820e-01,  ...,  1.2566e-01,
         -2.0207e-02,  9.8886e-02],
        [ 1.6956e+00,  1.1760e+00,  5.6003e-01,  ..., -6.7385e-01,
          2.1588e+00,  1.8154e+00],
        [ 2.3841e+01,  2.6361e+01,  1.2826e+01,  ...,  2.2554e+00,
          2.4942e+00,  1.7655e+00],
        ...,
        [ 2.7881e-01,  5.5533e-02,  2.9864e-01,  ...,  2.4708e-02,
          1.1570e-02,  2.6077e-02],
        [ 4.4138e+01,  2.5784e+01,  3.4246e+01,  ...,  1.7453e+01,
          1.6417e+01,  3.8603e+00],
        [ 9.7236e-01,  9.3319e-01,  9.2995e-01,  ...,  9.7565e-01,
          9.5715e-01,  9.2568e-01]])

In [11]:
ds_t = makeTorchDataset(feats_ak, ak.to_numpy(ak.flatten(tgt.genMatching)))

In [27]:
torch.tensor(ak.to_numpy(ak.flatten(tgt.genMatching)), dtype=torch.float32)[23]

tensor(0.)

In [25]:
np.nonzero(ak.to_numpy(ak.flatten(tgt.genMatching)))

(array([     22,      44,      51,      63,      75,     115,     129,
            167,     179,     236,     326,     356,     358,     361,
            386,    7459,    7488,    7490,    7493,    7516,   13765,
          13788,   13802,   13820,   13898,   21862,   21874,   21892,
          21972,   26062,   26101,   26167,   26168,   26170,   42641,
          42643,   42991,   48027,   49901,   49913,   49915,   58122,
          58124,   73376,   73382,   73432,   73488,   73550,   73567,
          73665,   73692,   73729,   73738,   90850,   90881,   90899,
          90909,   90923,   90943,   90959,   90987,   91003,   91013,
          91126,  105071,  105131,  105155,  105217,  105266,  105312,
         105323,  105356,  105387,  105388,  105427,  105434,  105439,
         105454,  105462,  105463,  124667,  124684,  124735,  124786,
         124913,  124923,  124927,  146473,  146596,  166175,  166229,
         166248,  166284,  166286,  182436,  182441,  182445,  204979,
      

In [29]:
ds_t[0:50]["features"][0]

tensor([[ 1.4991e-01,  1.6956e+00,  2.3841e+01, -2.8661e+00,  2.7054e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.1916e+01,  2.8193e-01,
          1.3713e-01,  1.3713e-01,  1.8231e+00,  5.3159e-02,  2.7881e-01,
          4.4138e+01,  9.7236e-01],
        [ 5.2284e-02,  1.1760e+00,  2.6361e+01, -2.7684e+00,  3.2958e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.8218e+01,  1.1699e-01,
          7.7512e-03,  7.7512e-03,  6.5022e-01,  1.0302e-01,  5.5533e-02,
          2.5784e+01,  9.3319e-01],
        [ 1.6820e-01,  5.6003e-01,  1.2826e+01, -2.8844e+00,  1.4293e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.0800e+01,  3.6202e-01,
          2.4707e-01,  2.4707e-01,  1.4768e+00,  2.1777e-01,  2.9864e-01,
          3.4246e+01,  9.2995e-01],
        [-8.6307e-02,  8.1876e-01,  1.6024e+01, -2.6298e+00,  2.2984e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  7.8261e+01,  3.2822e-01,
          1.9353e-01,  1.9353e-01,  1.7767e+00,  9.9170e-02,  3.1675e-01,
    

In [18]:
ds_t[0:5]["genmatching"][0]

tensor([False, False, False, False, False])

In [19]:
loss = torch.nn.BCELoss()
loss(torch.full((5,), 0.5), ds_t[0:5]["genmatching"][0])

RuntimeError: Found dtype Bool but expected Float

In [ ]:
device = "cpu"

dataset_torch = makeTorchDataset(makeFeatures(dataset_ak,  ['DeltaEtaBaryc', 'DeltaPhiBaryc', 'multi_en', 'multi_eta', 'multi_pt', 'seedEta','seedPhi','seedEn', 'seedPt', 'theta', 'theta_xz_seedFrame', 'theta_yz_seedFrame', 'theta_xy_cmsFrame', 'theta_yz_cmsFrame', 'theta_xz_cmsFrame', 'explVar', 'explVarRatio']),
                ak.to_numpy(ak.flatten(dataset_ak.genMatching)), device=device)